In [2]:
from generate import *
from evolve import *

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [3]:
model_name = "gpt2-xl-M0"
model_path = os.path.join(os.getcwd(), "models", model_name)
M0, tokenizer = load_model_and_tokenizer(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
pts_to_gen = 5000
D0 = generate_dataset(M0, tokenizer, pts_to_gen, keep_best=25)

100%|██████████| 5000/5000 [37:42<00:00,  2.21it/s]


In [9]:
datasave_path = os.path.join(os.getcwd(), "data", "D0-v2.pkl")
save_dataset(datasave_path, D0)

In [ ]:
dataset_name = "trial.pkl"
dataset_path = os.path.join(os.getcwd(), "data", dataset_name)
D0 = load_dataset(dataset_path)